In [44]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [28]:
df = pd.read_csv(r"D:\projects\ai_impact_on-_jobs\notebook\data\ai_impact_jobs_2010_2025.csv")

In [29]:
df.head()

,job_id,posting_year,country,region,city,company_name,company_size,industry,job_title,seniority_level,ai_mentioned,ai_keywords,ai_intensity_score,core_skills,ai_skills,salary_usd,salary_change_vs_prev_year_percent,automation_risk_score,reskilling_required,ai_job_displacement_risk,job_description_embedding_cluster,industry_ai_adoption_stage
0,836b4774-702e-49ef-93d3-2f255ce1e910,2018,Brazil,South America,London,NextGen Technologies,Small,Education,Policy Analyst,Lead,True,reinforcement learning,0.81,"Research, Project Management, Business Analysis",reinforcement learning,61586,12.68,0.11,True,Low,14,Growing
1,43699e93-7b15-4728-a4c6-9e41ff438a25,2015,UAE,Middle East,Singapore,Future Solutions,Medium,Energy,Data Scientist,Executive,False,NaN,0.04,"Research, SQL, Business Analysis, Python, Clou...",NaN,62045,-3.98,0.71,False,High,19,Emerging
2,fc9d1854-3cbf-4bab-90df-77304dfc59df,2016,Nepal,South Asia,Sydney,Future Analytics,Startup,Finance,Product Manager,Junior,False,NaN,0.15,"Statistics, Project Management, Cloud Computin...",NaN,27035,3.55,0.86,False,High,2,Emerging
3,05c1c7d3-2add-4919-91eb-f6c78bfe23d1,2015,Spain,Europe,Nairobi,Global Technologies,Large,Government,Data Scientist,Mid,False,NaN,0.19,"Cloud Computing, SQL, Project Management, Comm...",NaN,72894,-2.80,0.70,False,Low,15,Emerging
4,5e739937-d1b0-44d7-935c-7ebb3fc1f6e8,2014,Taiwan,East Asia,Sydney,Future Technologies,Small,Manufacturing,ML Engineer,Lead,False,NaN,0.11,"SQL, Python, Communication, Software Engineeri...",NaN,57215,0.85,0.87,False,High,13,Emerging


In [30]:
df["log_salary"] = np.log1p(df["salary_usd"])

In [31]:
df.shape


(5000, 23)

In [33]:
X = df.drop(columns=["salary_usd", "log_salary"], axis=1)

X.head()

,job_id,posting_year,country,region,city,company_name,company_size,industry,job_title,seniority_level,ai_mentioned,ai_keywords,ai_intensity_score,core_skills,ai_skills,salary_change_vs_prev_year_percent,automation_risk_score,reskilling_required,ai_job_displacement_risk,job_description_embedding_cluster,industry_ai_adoption_stage
0,836b4774-702e-49ef-93d3-2f255ce1e910,2018,Brazil,South America,London,NextGen Technologies,Small,Education,Policy Analyst,Lead,True,reinforcement learning,0.81,"Research, Project Management, Business Analysis",reinforcement learning,12.68,0.11,True,Low,14,Growing
1,43699e93-7b15-4728-a4c6-9e41ff438a25,2015,UAE,Middle East,Singapore,Future Solutions,Medium,Energy,Data Scientist,Executive,False,NaN,0.04,"Research, SQL, Business Analysis, Python, Clou...",NaN,-3.98,0.71,False,High,19,Emerging
2,fc9d1854-3cbf-4bab-90df-77304dfc59df,2016,Nepal,South Asia,Sydney,Future Analytics,Startup,Finance,Product Manager,Junior,False,NaN,0.15,"Statistics, Project Management, Cloud Computin...",NaN,3.55,0.86,False,High,2,Emerging
3,05c1c7d3-2add-4919-91eb-f6c78bfe23d1,2015,Spain,Europe,Nairobi,Global Technologies,Large,Government,Data Scientist,Mid,False,NaN,0.19,"Cloud Computing, SQL, Project Management, Comm...",NaN,-2.80,0.70,False,Low,15,Emerging
4,5e739937-d1b0-44d7-935c-7ebb3fc1f6e8,2014,Taiwan,East Asia,Sydney,Future Technologies,Small,Manufacturing,ML Engineer,Lead,False,NaN,0.11,"SQL, Python, Communication, Software Engineeri...",NaN,0.85,0.87,False,High,13,Emerging


In [35]:
y = df["log_salary"]
y

0       11.028206
1       11.035631
2       10.204925
3       11.196775
4       10.954589
          ...    
4995    10.629054
4996    10.521103
4997    11.310222
4998    11.588942
4999    10.484473
Name: log_salary, Length: 5000, dtype: float64

In [36]:
categorical_cols = X.select_dtypes(include=['object']).columns.to_list()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.to_list()

In [41]:
from scipy import sparse

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(
    handle_unknown="ignore",
    min_frequency=10,
    sparse_output=True
), categorical_cols)
    ]
)

X_transformed = preprocessor.fit_transform(X)

feature_names = preprocessor.get_feature_names_out()

X_transformed = pd.DataFrame(
    X_transformed.toarray() if hasattr(X_transformed, "toarray") else X_transformed,
    columns=feature_names
)

X_transformed

,num__posting_year,num__ai_intensity_score,num__salary_change_vs_prev_year_percent,num__automation_risk_score,num__job_description_embedding_cluster,cat__job_id_infrequent_sklearn,cat__country_Argentina,cat__country_Australia,cat__country_Bangladesh,cat__country_Brazil,cat__country_Canada,cat__country_Chile,cat__country_China,cat__country_Colombia,cat__country_Egypt,cat__country_France,cat__country_Germany,cat__country_Ghana,cat__country_India,cat__country_Indonesia,cat__country_Israel,cat__country_Italy,cat__country_Japan,cat__country_Jordan,cat__country_Kenya,cat__country_Malaysia,cat__country_Mexico,cat__country_Morocco,cat__country_Nepal,cat__country_Netherlands,cat__country_New Zealand,cat__country_Nigeria,cat__country_Pakistan,cat__country_Peru,cat__country_Philippines,cat__country_Poland,cat__country_Qatar,cat__country_Saudi Arabia,cat__country_Singapore,cat__country_South Africa,...,"cat__ai_keywords_deep learning, machine learning",cat__ai_keywords_generative AI,cat__ai_keywords_machine learning,"cat__ai_keywords_machine learning, NLP",cat__ai_keywords_reinforcement learning,"cat__ai_keywords_reinforcement learning, MLOps","cat__ai_keywords_reinforcement learning, NLP","cat__ai_keywords_reinforcement learning, deep learning",cat__ai_keywords_nan,cat__ai_keywords_infrequent_sklearn,cat__core_skills_infrequent_sklearn,cat__ai_skills_LLMs,cat__ai_skills_MLOps,"cat__ai_skills_MLOps, reinforcement learning",cat__ai_skills_NLP,"cat__ai_skills_NLP, MLOps","cat__ai_skills_NLP, computer vision","cat__ai_skills_NLP, reinforcement learning",cat__ai_skills_computer vision,"cat__ai_skills_computer vision, MLOps","cat__ai_skills_computer vision, NLP",cat__ai_skills_deep learning,"cat__ai_skills_deep learning, MLOps","cat__ai_skills_deep learning, NLP","cat__ai_skills_deep learning, machine learning",cat__ai_skills_generative AI,cat__ai_skills_machine learning,"cat__ai_skills_machine learning, NLP",cat__ai_skills_reinforcement learning,"cat__ai_skills_reinforcement learning, MLOps","cat__ai_skills_reinforcement learning, NLP","cat__ai_skills_reinforcement learning, deep learning",cat__ai_skills_nan,cat__ai_skills_infrequent_sklearn,cat__ai_job_displacement_risk_High,cat__ai_job_displacement_risk_Low,cat__ai_job_displacement_risk_Medium,cat__industry_ai_adoption_stage_Emerging,cat__industry_ai_adoption_stage_Growing,cat__industry_ai_adoption_stage_Mature
0,0.103620,1.794032,1.619403,-1.904845,0.787552,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,-0.543196,-0.848854,-1.172091,0.495333,1.657392,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-0.327591,-0.471298,0.089611,1.095377,-1.300062,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,-0.543196,-0.334006,-0.974374,0.455330,0.961520,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,-0.758801,-0.608591,-0.362792,1.135380,0.613585,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.

In [42]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

In [45]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.0862
- Mean Absolute Error: 0.0742
- R2 Score: 0.9741
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0892
- Mean Absolute Error: 0.0770
- R2 Score: 0.9736


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.5357
- Mean Absolute Error: 0.4463
- R2 Score: 0.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.5493
- Mean Absolute Error: 0.4619
- R2 Score: -0.0001


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.0862
- Mean Absolute Error: 0.0742
- R2 Score: 0.9741
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0892
- Mean Absolute Error: 0.0770
- R2 Score: 0.9736


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.2314
- Mean Absolute Error: 0.1722
- R2 Score: 0.8135
----------------------

In [ ]:
# One-hot encoding initially caused dimensional explosion.
# I controlled it using infrequent category handling, feature selection, and model-aware encoding.
# Linear and tree-based models both validated the signal, with CatBoost offering the best balance of performance and robustness.